In [1]:
!pip install kaggle

In [2]:
creds = {"username":"ahmadkaddoura","key":"aca490a33efaa2e266410a1833167d66"}

In [3]:
import zipfile,kaggle
from pathlib import Path

path = Path('us-patent-phrase-to-phrase-matching')
kaggle.api.competition_download_cli(str(path))
zipfile.ZipFile(f'{path}.zip').extractall(path)

us-patent-phrase-to-phrase-matching.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
!pip install -q datasets

In [5]:
import pandas as pd


In [6]:
df = pd.read_csv('../fastai/us-patent-phrase-to-phrase-matching/train.csv')

In [7]:
df.describe(include='object')

,id,anchor,target,context
count,36473,36473,36473,36473
unique,36473,733,29340,106
top,37d61fd2272659b1,component composite coating,composition,H01
freq,1,152,24,2186


In [8]:
#df['input'] = f'Text1: {df.context} ; Text2: {df.target} ; Anc1: {df.anchor}'\
df['input'] = f'Text1: '+ df.context + ' Text2: ' + df.target +' ; Anc1: ' + df.anchor

In [9]:
df.input.head()

0    Text1: A47 Text2: abatement of pollution ; Anc...
1    Text1: A47 Text2: act of abating ; Anc1: abate...
2    Text1: A47 Text2: active catalyst ; Anc1: abat...
3    Text1: A47 Text2: eliminating process ; Anc1: ...
4    Text1: A47 Text2: forest region ; Anc1: abatement
Name: input, dtype: object

In [10]:
from datasets import Dataset,DatasetDict

ds = Dataset.from_pandas(df)
ds

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'score', 'input'],
    num_rows: 36473
})

In [11]:
model_nm = 'microsoft/deberta-v3-small'

In [12]:
!pip install transformers

In [13]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
tokz = AutoTokenizer.from_pretrained(model_nm)

k:\AI\ak_tensorflow_collab_training\.conda\Lib\site-packages\transformers\convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [14]:
tokz.tokenize("hello my dear an Sally, what have you eaten for breakfast today?")

['▁hello',
 '▁my',
 '▁dear',
 '▁an',
 '▁Sally',
 ',',
 '▁what',
 '▁have',
 '▁you',
 '▁eaten',
 '▁for',
 '▁breakfast',
 '▁today',
 '?']

In [15]:
def tok_func(x) : return tokz(x['input'])

In [16]:
toks_ds = ds.map(tok_func,batched=True)

Map:   0%|          | 0/36473 [00:00<?, ? examples/s]

In [17]:
row = toks_ds[0]
row['input'],row['input_ids']

('Text1: A47 Text2: abatement of pollution ; Anc1: abatement',
 [1,
  7655,
  435,
  294,
  336,
  5753,
  7655,
  445,
  294,
  47284,
  265,
  6435,
  2600,
  816,
  1207,
  435,
  294,
  47284,
  2])

In [18]:
tokz.vocab['pollution']

99050

In [19]:
toks_ds=toks_ds.rename_columns({'score':'labels'})

In [20]:
eval_df = pd.read_csv('us-patent-phrase-to-phrase-matching/train.csv')
eval_df.describe(include='object')

,id,anchor,target,context
count,36473,36473,36473,36473
unique,36473,733,29340,106
top,37d61fd2272659b1,component composite coating,composition,H01
freq,1,152,24,2186


In [21]:
dds = toks_ds.train_test_split(0.25,seed =10)
dds

DatasetDict({
    train: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 27354
    })
    test: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9119
    })
})

In [39]:


eval_df['input'] = 'TEXT1: ' + eval_df.context + '; TEXT2: ' + eval_df.target + '; ANC1: ' + eval_df.anchor
eval_ds = Dataset.from_pandas(eval_df).map(tok_func, batched=True)



Map:   0%|          | 0/36473 [00:00<?, ? examples/s]

In [40]:
from transformers import TrainingArguments,Trainer
import math


In [23]:
!pip install accelerate -U


In [41]:
bs= 128
epochs = 5
lr = 8e-5

In [43]:
args = TrainingArguments('outputs',
                         learning_rate=lr,
                         warmup_ratio=.1,
                         lr_scheduler_type='cosine', 
                         fp16=True,
                         evaluation_strategy="epoch",
                         per_device_eval_batch_size=bs*2,
                         per_device_train_batch_size=bs,
                         num_train_epochs=epochs,
                         weight_decay=.01,
                         report_to="none")

In [44]:
def corr(x,y): return np.corrcoef(x,y)[0][1]
def corr_d(eval_pred): return {'pearson': corr(*eval_pred)}

In [45]:
model = AutoModelForSequenceClassification.from_pretrained(model_nm,num_labels=1)
trainer = Trainer(model,args,train_dataset=dds['train'],eval_dataset=dds['test'],tokenizer=tokz,compute_metrics=corr_d)

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.weight', 'classifier.bias', 'pooler.dense.weight', 'pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [46]:
import numpy as np

In [47]:
trainer.train();

  0%|          | 0/1070 [00:00<?, ?it/s]

  0%|          | 0/36 [00:00<?, ?it/s]

{'eval_loss': 0.024191400036215782, 'eval_pearson': 0.8013640431969737, 'eval_runtime': 0.9479, 'eval_samples_per_second': 9619.995, 'eval_steps_per_second': 37.978, 'epoch': 1.0}


  0%|          | 0/36 [00:00<?, ?it/s]

{'eval_loss': 0.023879537358880043, 'eval_pearson': 0.8239720911991398, 'eval_runtime': 0.9229, 'eval_samples_per_second': 9880.604, 'eval_steps_per_second': 39.007, 'epoch': 2.0}
{'loss': 0.0314, 'learning_rate': 5.1388762323647767e-05, 'epoch': 2.34}


...

{'eval_loss': 0.02206900715827942, 'eval_pearson': 0.8346824154723499, 'eval_runtime': 0.9628, 'eval_samples_per_second': 9471.167, 'eval_steps_per_second': 37.39, 'epoch': 3.0}


  0%|          | 0/36 [00:00<?, ?it/s]

{'eval_loss': 0.022259224206209183, 'eval_pearson': 0.839742604711239, 'eval_runtime': 0.9557, 'eval_samples_per_second': 9541.215, 'eval_steps_per_second': 37.667, 'epoch': 4.0}
{'loss': 0.0124, 'learning_rate': 1.038448832995451e-06, 'epoch': 4.67}


  0%|          | 0/36 [00:00<?, ?it/s]

{'eval_loss': 0.021980004385113716, 'eval_pearson': 0.8398912025845181, 'eval_runtime': 0.9642, 'eval_samples_per_second': 9457.135, 'eval_steps_per_second': 37.335, 'epoch': 5.0}
{'train_runtime': 65.3835, 'train_samples_per_second': 2091.813, 'train_steps_per_second': 16.365, 'train_loss': 0.02115028066055797, 'epoch': 5.0}


In [48]:
preds = trainer.predict(eval_ds).predictions.astype(float)
preds

  0%|          | 0/143 [00:00<?, ?it/s]

array([[0.55957031],
       [0.61279297],
       [0.25048828],
       ...,
       [0.54736328],
       [0.74658203],
       [0.5390625 ]])

In [49]:
preds = np.clip(preds, 0, 1)
preds

array([[0.55957031],
       [0.61279297],
       [0.25048828],
       ...,
       [0.54736328],
       [0.74658203],
       [0.5390625 ]])

In [50]:
import datasets

submission = datasets.Dataset.from_dict({
    'id': eval_ds['id'],
    'score': preds
})

submission.to_csv('submission.csv', index=False)

Creating CSV from Arrow format:   0%|          | 0/37 [00:00<?, ?ba/s]

1087728